<a href="https://colab.research.google.com/github/KrDevanshu06/dynamic-parking-pricing-capstone/blob/main/%5BPublic%5DDynamic_Pricing_Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic Pricing for Urban Parking Lots

**Summer Analytics 2025 - Capstone Project**

This notebook implements three real-time dynamic pricing models for parking lots using only:
- `numpy`, `pandas` for logic
- `Bokeh` for real-time visualization
- `Pathway` (to be integrated later) for real-time data streaming simulation

**Models Implemented:**
1. Baseline Linear Pricing
2. Demand-Based Dynamic Pricing
3. Competitive Pricing with Location Intelligence & Rerouting Suggestions

All price changes are smooth, bounded, and explainable as per the project requirements.


## 🔵 **Cell 1: Mandatory Installations**

In [ ]:
!pip install pathway bokeh --quiet


## 🔵 **Cell 2: Imports & Setup**


In [ ]:
import pandas as pd
import numpy as np
import pathway as pw
import time
from bokeh.plotting import figure, output_notebook, show
from bokeh.io import push_notebook

output_notebook()

## 🔵 **Cell 3: Manual Haversine Distance Function (Numpy Only)**


In [ ]:
def haversine_np(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c


## 🔵 **Cell 4: Dataset Preparation**

In [ ]:
# Load dataset
df = pd.read_csv('dataset.csv')

# Combine date and time for timestamp, fix date parsing with dayfirst=True
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)


# Sort by lot and timestamp
df.sort_values(['SystemCodeNumber', 'Timestamp'], inplace=True)
df.reset_index(drop=True, inplace=True)

# Extract lot IDs and coordinates
parking_lots = df['SystemCodeNumber'].unique()
lot_locations = df.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].first().to_dict('index')



## 🔵 **Cell 5: Global Parameters**


In [ ]:
# Pricing boundaries
BASE_PRICE = 10
PRICE_MIN = BASE_PRICE * 0.5
PRICE_MAX = BASE_PRICE * 2
PROXIMITY_RADIUS = 0.5  # km

# Demand function weights
vehicle_weight = {'car': 1, 'bike': 0.5, 'truck': 1.5}
traffic_map = {'low': 0, 'medium': 0.5, 'high': 1}
alpha, beta, gamma, delta, epsilon = 0.6, 0.3, 0.2, 0.5, 0.4
lambda_ = 0.5
max_queue = 10
max_vehicle_weight = max(vehicle_weight.values())


# ⚡ Real-Time Pricing with Pathway Streaming


## 🔵 **Cell 6: Pathway Streaming Simulation Setup**


In [ ]:
# Pathway streaming input schema
class ParkingStream(pw.Schema):
    SystemCodeNumber: str
    Latitude: float
    Longitude: float
    Capacity: int
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: int
    Timestamp: str


## 🔵 **Cell 7: Real-Time Bokeh Plot Setup**


In [ ]:
# Initialize price tracker
price_tracker = {lot: [BASE_PRICE] for lot in parking_lots}

# Bokeh color map
color_list = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'grey', 'cyan', 'magenta', 'olive', 'navy', 'lime', 'teal']
color_map = {lot: color for lot, color in zip(parking_lots, color_list)}

# Bokeh plot (Updated attributes)
p = figure(title="Real-Time Dynamic Pricing (Pathway Stream)", x_axis_type='datetime', height=400, width=800)

# Create lines for each parking lot
line_dict = {
    lot: p.line([], [], legend_label=lot, color=color_map[lot], line_width=2)
    for lot in parking_lots
}

p.legend.click_policy = "hide"
show(p, notebook_handle=True)


## 🔵 **Cell 8: Pathway Data Stream Simulation & Pricing Logic**

In [ ]:
df_sorted = df.sort_values('Timestamp')

for idx, row in df_sorted.iterrows():
    lot = row['SystemCodeNumber']
    coords = (row['Latitude'], row['Longitude'])
    cap = row['Capacity']
    occ = row['Occupancy']
    queue = row['QueueLength']
    special = row['IsSpecialDay']
    vehicle = row['VehicleType']
    traffic = traffic_map.get(row['TrafficConditionNearby'], 0.5)
    timestamp = row['Timestamp']

    weight = vehicle_weight.get(vehicle, 1.0)

    raw_demand = (
        alpha * (occ / cap) +
        beta * queue -
        gamma * traffic +
        delta * special +
        epsilon * weight
    )

    max_demand = alpha + beta * max_queue + delta + epsilon * max(vehicle_weight.values())
    normalized_demand = np.clip(raw_demand / max_demand, 0, 1)

    new_price = BASE_PRICE * (1 + lambda_ * normalized_demand)

    nearby = []
    for comp in parking_lots:
        if comp == lot:
            continue
        lat1, lon1 = coords
        lat2, lon2 = lot_locations[comp]['Latitude'], lot_locations[comp]['Longitude']
        dist = haversine_np(lat1, lon1, lat2, lon2)
        if dist <= PROXIMITY_RADIUS:
            nearby.append(comp)

    avg_comp_price = np.mean([price_tracker[comp][-1] for comp in nearby]) if nearby else BASE_PRICE

    if occ >= cap and nearby and avg_comp_price < new_price:
        print(f"[{timestamp.time()}] Suggest rerouting from {lot} to cheaper nearby lots.")
        new_price = avg_comp_price - 0.5
    elif nearby and avg_comp_price > new_price:
        new_price = min(avg_comp_price + 0.5, PRICE_MAX)

    new_price = np.clip(new_price, PRICE_MIN, PRICE_MAX)

    price_tracker[lot].append(new_price)

    line = line_dict[lot]
    new_x = list(line.data_source.data['x']) + [timestamp]
    new_y = list(line.data_source.data['y']) + [new_price]
    line.data_source.data = {'x': new_x, 'y': new_y}

    push_notebook()
    time.sleep(0.05)

print("✅ Real-time simulation completed.")


✅ Real-time simulation completed.
